<a href="https://colab.research.google.com/github/c-kartik/BERT_NER/blob/main/BERT_w_NER_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64",

In [2]:
!pip install --upgrade datasets

In [3]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizerFast, BertModel, TrainingArguments, Trainer, DataCollatorForTokenClassification, AutoModelForTokenClassification
import datasets
import evaluate
import json

In [5]:
# 1. Loading Dataset and Labels

dataset = datasets.load_dataset("conll2003")

# Get the list of label names for each task.
ner_label_list = dataset["train"].features["ner_tags"].feature.names
pos_label_list = dataset["train"].features["pos_tags"].feature.names

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [6]:
# 2. Initialize Tokenizer

# Using the fast BERT tokenizer (cased) so that pre-tokenized words are handled correctly.
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Tokenizes inputs while aligning word-level NER and POS labels to (possibly split) tokens.
    Subword tokens are either given the same label (if label_all_tokens=True) or masked with -100.
    """
    tokenized_inputs = tokenizer(examples["tokens"],
                                 truncation=True,
                                 is_split_into_words=True)

    all_ner_labels = []
    all_pos_labels = []

    # Iterate over the batch using the number of examples in input_ids.
    for i in range(len(tokenized_inputs["input_ids"])):
        # Get the mapping from token indices to original word indices for the i-th example.
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        ner_label_ids = []
        pos_label_ids = []

        for word_idx in word_ids:
            # When word_idx is None, it is a special token (e.g., CLS or SEP)
            if word_idx is None:
                ner_label_ids.append(-100)
                pos_label_ids.append(-100)
            # For the first token of a given word, assign the real label.
            elif word_idx != previous_word_idx:
                ner_label_ids.append(examples["ner_tags"][i][word_idx])
                pos_label_ids.append(examples["pos_tags"][i][word_idx])
            # For subsequent tokens of the same word:
            else:
                ner_label_ids.append(examples["ner_tags"][i][word_idx] if label_all_tokens else -100)
                pos_label_ids.append(examples["pos_tags"][i][word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        all_ner_labels.append(ner_label_ids)
        all_pos_labels.append(pos_label_ids)

    tokenized_inputs["ner_labels"] = all_ner_labels
    tokenized_inputs["pos_labels"] = all_pos_labels
    return tokenized_inputs

# Apply the tokenization function to the entire dataset.
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
##############################################
# 4. Define the Multi-Task Model
##############################################

class MultiTaskBertForTokenClassification(nn.Module):
    """
    A custom model that extends BERT with two token classification heads:
      - One for NER (e.g., PER, LOC, ORG, MISC)
      - One for POS tagging (e.g., nouns, verbs, adjectives, etc.)
    """
    def __init__(self, model_name, num_ner_labels, num_pos_labels):
        super(MultiTaskBertForTokenClassification, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.ner_classifier = nn.Linear(self.bert.config.hidden_size, num_ner_labels)
        self.pos_classifier = nn.Linear(self.bert.config.hidden_size, num_pos_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, ner_labels=None, pos_labels=None):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)

        ner_logits = self.ner_classifier(sequence_output)
        pos_logits = self.pos_classifier(sequence_output)

        loss = None
        if ner_labels is not None and pos_labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            # Compute loss for each task independently, then sum them.
            ner_loss = loss_fct(ner_logits.view(-1, ner_logits.shape[-1]), ner_labels.view(-1))
            pos_loss = loss_fct(pos_logits.view(-1, pos_logits.shape[-1]), pos_labels.view(-1))
            loss = ner_loss + pos_loss

        return {"loss": loss, "ner_logits": ner_logits, "pos_logits": pos_logits}

# Instantiate the multi-task model using the number of labels for each task.
num_ner_labels = len(ner_label_list)
num_pos_labels = len(pos_label_list)
model = MultiTaskBertForTokenClassification("bert-base-cased", num_ner_labels, num_pos_labels)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [9]:
##############################################
# 5. Create a Custom Data Collator (Fixed Version)
##############################################

def data_collator(features):
    """
    Pads the inputs as well as the ner_labels and pos_labels.

    This version pops the labels from each example so the tokenizer.pad method
    only handles the standard fields (like input_ids, attention_mask, etc.). The labels
    are then padded manually.
    """
    # Extract label fields and remove them from the features dict.
    ner_labels = [f.pop("ner_labels") for f in features]
    pos_labels = [f.pop("pos_labels") for f in features]

    # Use the tokenizer to pad the remaining fields.
    batch = tokenizer.pad(features, return_tensors="pt")

    # Manually pad the label sequences using -100 as the padding index.
    batch["ner_labels"] = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(label_seq) for label_seq in ner_labels],
        batch_first=True,
        padding_value=-100,
    )
    batch["pos_labels"] = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(label_seq) for label_seq in pos_labels],
        batch_first=True,
        padding_value=-100,
    )

    return batch

In [10]:
# 6. Define Evaluation Metrics

# Load the seqeval metric for NER evaluation.
ner_metric = evaluate.load("seqeval") # seqeval is usde for sequence labelling

def compute_metrics(eval_pred):
    """
    Computes metrics for both tasks:
      - For NER: uses the seqeval metric (precision, recall, F1)
      - For POS: calculates a simple token-level accuracy (ignoring padding tokens)
    """
    predictions, labels = eval_pred
    ner_logits = predictions["ner_logits"]
    pos_logits = predictions["pos_logits"]
    ner_labels = labels["ner_labels"]
    pos_labels = labels["pos_labels"]

    # Convert logits to predicted label IDs.
    ner_pred_ids = np.argmax(ner_logits, axis=-1)
    pos_pred_ids = np.argmax(pos_logits, axis=-1)

    true_ner_labels = []
    pred_ner_labels = []
    true_pos_labels = []
    pred_pos_labels = []

    # Loop through each sentence to remove padding (-100) and map indices to label names.
    for i in range(len(ner_labels)):
        ner_true = []
        ner_pred = []
        pos_true = []
        pos_pred = []
        for j in range(len(ner_labels[i])):
            if ner_labels[i][j] != -100:
                ner_true.append(ner_label_list[ner_labels[i][j]])
                ner_pred.append(ner_label_list[ner_pred_ids[i][j]])
            if pos_labels[i][j] != -100:
                pos_true.append(pos_label_list[pos_labels[i][j]])
                pos_pred.append(pos_label_list[pos_pred_ids[i][j]])
        true_ner_labels.append(ner_true)
        pred_ner_labels.append(ner_pred)
        true_pos_labels.append(pos_true)
        pred_pos_labels.append(pos_pred)

    # Compute NER metrics.
    ner_results = ner_metric.compute(predictions=pred_ner_labels, references=true_ner_labels)

    # Compute POS token-level accuracy.
    total_tokens = 0
    correct_tokens = 0
    for true_seq, pred_seq in zip(true_pos_labels, pred_pos_labels):
        for t, p in zip(true_seq, pred_seq):
            total_tokens += 1
            if t == p:
                correct_tokens += 1
    pos_accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0.0

    return {"ner": ner_results, "pos_accuracy": pos_accuracy}

In [11]:
# 7. Set Up the Trainer and Training Arguments

training_args = TrainingArguments(
    output_dir="multi_task_ner_pos",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

<ipython-input-11-b11c8d3aa49f>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
# 8. Training

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kartik-c612 (kartik-c612-university-of-british-columbia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.064200
1000,0.408300
1500,0.320600
2000,0.278100
2500,0.246400


TrainOutput(global_step=2634, training_loss=0.45164414943848585, metrics={'train_runtime': 783.3838, 'train_samples_per_second': 53.771, 'train_steps_per_second': 3.362, 'total_flos': 0.0, 'train_loss': 0.45164414943848585, 'epoch': 3.0})

In [13]:
# 9. Save the Model and Update Configuration

import os

model_save_path = "multi_task_ner_pos_model"
os.makedirs(model_save_path, exist_ok=True)  # Create directory if it does not exist


# Save model weights and tokenizer
torch.save(model.state_dict(), model_save_path + "/pytorch_model.bin")
tokenizer.save_pretrained(model_save_path)

# Save additional configuration details (such as label mappings) into the config.
config = model.bert.config.to_dict()
config["id2label_ner"] = {i: label for i, label in enumerate(ner_label_list)}
config["label2id_ner"] = {label: i for i, label in enumerate(ner_label_list)}
config["id2label_pos"] = {i: label for i, label in enumerate(pos_label_list)}
config["label2id_pos"] = {label: i for i, label in enumerate(pos_label_list)}
with open(os.path.join(model_save_path, "config.json"), "w") as f:
    json.dump(config, f)

In [14]:
def run_inference(text):
    """
    Runs the multi-task model on an input text string and returns both NER and POS predictions.
    The function tokenizes the input and aligns the predictions based on the word_ids.
    """
    # Determine the device the model is on (or move it to one if you prefer)
    device = next(model.parameters()).device

    # Tokenize the input text.
    tokenized_input = tokenizer(text, return_tensors="pt", is_split_into_words=False)

    # Move tokenized tensors to the same device as the model.
    tokenized_input = {k: v.to(device) for k, v in tokenized_input.items()}

    # Obtain the model outputs.
    outputs = model(**tokenized_input)
    ner_logits = outputs["ner_logits"].detach().cpu().numpy()
    pos_logits = outputs["pos_logits"].detach().cpu().numpy()

    # Get predicted label IDs.
    ner_pred_ids = np.argmax(ner_logits, axis=-1)[0]
    pos_pred_ids = np.argmax(pos_logits, axis=-1)[0]

    # Convert input ids to tokens.
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"].detach().cpu()[0])

    # Retrieve the word ids for alignment.
    word_ids = tokenizer(text, return_tensors="pt", is_split_into_words=False).word_ids(batch_index=0)

    # Aggregate predictions for each original word.
    ner_results = []
    pos_results = []
    last_word = None
    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        # To avoid repeating predictions for subword tokens, only record the first token for each word.
        if word_idx != last_word:
            ner_results.append({
                "entity": ner_label_list[ner_pred_ids[idx]],
                "word": tokens[idx]
            })
            pos_results.append({
                "token": tokens[idx],
                "pos": pos_label_list[pos_pred_ids[idx]]
            })
            last_word = word_idx
    return ner_results, pos_results

# Example usage similar to the original NER pipeline demonstration.
example = "Bill Gates is the Founder of Microsoft"
ner_out, pos_out = run_inference(example)
print("NER Output:")
print(ner_out)
print("\nPOS Output:")
print(pos_out)

NER Output:
[{'entity': 'B-PER', 'word': 'Bill'}, {'entity': 'I-PER', 'word': 'Gates'}, {'entity': 'O', 'word': 'is'}, {'entity': 'O', 'word': 'the'}, {'entity': 'O', 'word': 'Founder'}, {'entity': 'O', 'word': 'of'}, {'entity': 'B-ORG', 'word': 'Microsoft'}]

POS Output:
[{'token': 'Bill', 'pos': 'NNP'}, {'token': 'Gates', 'pos': 'NNP'}, {'token': 'is', 'pos': 'VBZ'}, {'token': 'the', 'pos': 'DT'}, {'token': 'Founder', 'pos': 'NNP'}, {'token': 'of', 'pos': 'IN'}, {'token': 'Microsoft', 'pos': 'NNP'}]
